# Mini Project 3

In [ ]:
import numpy as np
import pandas as pd
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

## Task 1: Acquire the data

load dataset using tensorflow library

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train.shape
print(len(X_train))

4431872/4422102 [==============================] - 0s 0us/step
60000


We start by transforming the training and test sets into a 1D array. 



In [ ]:
y_train_arr = []
for i in range(len(y_train)):
  y_train_zeros = np.zeros(9)
  a = np.insert(y_train_zeros, y_train[i], 1)
  y_train_arr.append(a)


In [ ]:
X_train = np.copy(X_train)
X_train_f = X_train.astype(np.float64)
X_train_f -= np.mean(X_train_f)
X_train_f /= np.std(X_train_f)

X_test = np.copy(X_test)
X_test_f = X_test.astype(np.float64)
X_test_f -= np.mean(X_test_f)
X_test_f /= np.std(X_test_f)

X_train_vect = X_train.reshape(X_train_f.shape[0], (X_train_f.shape[1]*X_train_f.shape[2]))
X_test_vect = X_test.reshape(X_test_f.shape[0], (X_test_f.shape[1]*X_test_f.shape[2]))
from matplotlib import pyplot
pyplot.imshow(X_train[0])

NameError: ignored

## Task 2: Implement a Multilayer Perceptron

Multilayer Perceptron Class (to be finished)

In [ ]:
#define activation functions and their derivatives
def relu(x):
  return 0.0 if x < 0.0 else x
relu_v = np.vectorize(relu)
def relu_deriv(x):
  return 0.0 if x < 0.0 else 1.0
relu_deriv_v = np.vectorize(relu_deriv)
def leaky_relu(x):
  return 0.1 * x if x < 0 else x
leaky_relu_v = np.vectorize(leaky_relu)
def leaky_deriv(x):
  return 0.1 if x < 0 else 1
leaky_deriv_v = np.vectorize(leaky_deriv)
def tanh(x):
  return np.tanh(x)
tanh_v = np.vectorize(tanh)
def tanh_deriv(x):
  return 0.1 - np.tanh(x)**2
tanh_deriv_v = np.vectorize(tanh_deriv)

def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum(axis=0)

def cross_entropy(yh, y):
    if y == 1.0:
      return -np.log(yh)
    else:
      return -np.log(1 - yh)

def log_softmax(x):
    c = x.max()
    logsumexp = np.log(np.exp(x - c).sum())
    return x - c - logsumexp


class MultilayerPerceptron:
    
    def __init__(self, activation_function, activation_derivative, num_hidden_layers, num_units_hidden_layers):
      self.activation_function = activation_function
      self.activation_derivative = activation_derivative
      self.num_hidden_layers = num_hidden_layers
      self.num_units_hidden_layers = num_units_hidden_layers
      D = [28,28]
      C = 9
      # initialize weights and bias' randomly
      hidden_layers = []
      for i in range(num_hidden_layers):
        hidden_layers.append(num_units_hidden_layers)
      
      layers = [2] + hidden_layers + [1]

      weights = []
      bias = []
      for i in range(len(layers) - 1):
        w = np.random.randn(layers[i], layers[i + 1]) * 0.01
        weights.append(w)
        b = np.full(layers[i + 1], 0.1)
        bias.append(b)
      self.weights = weights
      self.bias = bias


      activations = []
      for i in range(len(layers)):
        a = np.zeros(layers[i])
        activations.append(a)
      self.activations = activations

      derivatives = [] 
      for i in range(len(layers) - 1):
        d = np.zeros((layers[i], layers[i+1]))
        derivatives.append(d)
      self.derivatives = derivatives

    def forward_propagate(self, inputs):
      activations = inputs
      self.activations[0] = inputs

      for i, w in enumerate(self.weights):
        b = self.bias[i]
        
        net_inputs = np.dot(activations, w) 

        # print(net_inputs)

        # apply activation function
        activations = self.activation_function(net_inputs)
        # if(i == len(self.weights) - 1):
        #   activations = softmax(activations)

        # print(activations)
        self.activations[i+1] = activations

      return activations

    def back_propagate(self, error):
      for i in reversed(range(len(self.derivatives))):
        activations = self.activations[i+1]
        delta = error * self.activation_derivative(activations) 
        delta_reshaped = delta.reshape(delta.shape[0], -1).T     #1D array to 2D array with single row
        current_activations = self.activations[i] 
        current_activations_reshaped = current_activations.reshape(current_activations.shape[0], -1) #1D array to 2D array with multiple rows

        self.derivatives[i] = np.dot(current_activations_reshaped, delta_reshaped)
        error = np.dot(delta, self.weights[i].T)
      


    # need to add bias
    def gradient_descent(self, learning_rate):
      for i in range(len(self.weights)):
        weigths = self.weights[i]
        derivatives = self.derivatives[i]
        weigths += derivatives * learning_rate

    def fit(self, inputs, targets, learning_rate, num_of_iterations):

      for i in range(num_of_iterations):
        # iterate through every input and its target one by one
        for x, y in zip(inputs, targets):
          #forward propagate
          output = self.forward_propagate(x)

          #calculate error
          # error = 0
          # for i in range(len(output)):
          #   output[i] = max(1e-9, output[i])

          # error = -np.sum(y * np.log(output))
          # error = np.log(error)
          # for i in range(len(y)):
          #   val = cross_entropy(output[i], y[i])
          #   error += val

          error = y - output

          #back propagate
          self.back_propagate(error)

          self.gradient_descent(learning_rate)
      
    def predict(self, inputs):
      # predictions = []
      for x in inputs:
        prediction = self.forward_propagate(x)    
        print(prediction)
      #   predictions.append(prediction)
      # return predictions

    def evaluate_acc(self, true_targets, predicted_targets):
      errors = []
      for true, pred in zip(true_targets, predicted_targets):
        error = true - pred
        errors.append(error)
      
      accuracy = np.mean(errors)
      return accuracy


In [ ]:
#test 
mlp = MultilayerPerceptron(relu_v, relu_deriv_v, 2, 128)

mlp.fit(X_train_vect[0:8], y_train_arr[0:8], 0.01, 5)



[0.09189068 0.09576597 0.09189068 0.09362778 0.09189068 0.09307197
 0.09189068 0.11770435 0.14037655 0.09189068]
2.3871557229002267
[1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e+00 1.e-09]
20.72326583694641
[1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e+00 1.e-09]
20.72326583694641
[1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e+00 1.e-09]
20.72326583694641
[1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 5.e-01 5.e-01]
20.72326583694641
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
2.3025850929940455
[1.25e-01 1.25e-01 1.25e-01 1.25e-01 1.25e-01 1.25e-01 1.25e-01 1.25e-01
 1.00e-09 1.00e-09]
2.0794415416798357
[1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09]
20.72326583694641
[1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09]
20.72326583694641
[1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09]
20.72326583694641
[1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1.e-09 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in subtract


In [ ]:
print(relu_v(-5))

0.0


In [ ]:
# predictions do not work and give NAN 
output = mlp.forward_propagate(X_test_vect[0])
print(output)
print(y_test[0])

[3.13148535e+20 3.19816838e+26 1.37065676e+26 4.23188602e+22
 7.28038904e+22 9.25516013e+26 1.77814649e+26 2.12401458e+21
 2.84044379e+25 1.22440152e+27 3.13075925e+26 2.17924804e+26
 7.40114087e+26 1.34004616e+27 1.88732163e+22 3.04362517e+27
 5.89393557e+26 6.51230610e+25 1.63550560e+22 0.00000000e+00
 1.02765935e+27 9.56784536e+26 0.00000000e+00 1.39443727e+27
 8.08627773e+26 5.76486198e+25 9.91134163e+22 2.44955174e+22
 1.64599333e+27 3.81989285e+16 3.10080551e+26 1.14545021e+27
 3.81335943e+22 5.00501730e+22 3.53027172e+22 6.51173187e+25
 9.72157920e+26 1.43347954e+17 2.53426384e+25 2.17700692e+25
 1.66560694e+23 6.20073342e+12 1.40060312e+23 2.28721255e+22
 0.00000000e+00 1.10640692e+27 0.00000000e+00 4.46862798e+25
 1.25451531e+26 1.40484333e+23 5.07688325e+25 9.58971819e+25
 4.49812160e+22 0.00000000e+00 1.10111101e+23 0.00000000e+00
 2.62293423e+22 1.73896358e+27 5.95101797e+22 1.54347327e+27
 6.84417185e+26 1.61660066e+23 1.88464147e+27 1.34662287e+22
 3.32897027e+16 1.250886

In [ ]:
from random import random
items = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
targets = np.array([[i[0] + i[1]] for i in items])

mlp2 = MultilayerPerceptron(relu_v, relu_deriv_v, 2, 5)

mlp2.fit(items, targets, 0.01, 50)

# create dummy data
inputss = np.array([0.3, 0.1])
target = np.array([0.4])

# get a prediction
output = mlp2.forward_propagate(inputss)

print()
print("Our network believes that {} + {} is equal to {}".format(inputss[0], inputss[1], output[0]))


Our network believes that 0.3 + 0.1 is equal to 4.68012593395759e-06


In [ ]:
 model = Sequential()
 model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1),    
  activation='relu'))
 model.add(MaxPooling2D())
 model.add(Dropout(0.2))
 model.add(Flatten())
 model.add(Dense(1, activation='relu'))
 model.add(Dense(1, activation='softmax'))

 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

Epoch 1/10
109/300 [=========>....................] - ETA: 15s - loss: 0.0000e+00 - accuracy: 0.0987

KeyboardInterrupt: ignored

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="Same", activation="relu", input_shape=[28, 28, 1]))
#model.add(MaxPooling2D())
#model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding="Same", activation="relu"))
#model.add(MaxPooling2D())
#model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)


Epoch 1/10


ValueError: ignored